In [ ]:
!pip install gradio transformers pillow torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-

In [ ]:
import gradio as gr
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
from PIL import Image
import torch.nn as nn
import matplotlib.pyplot as plt
import torch
import numpy as np

# Cargar el procesador y el modelo
processor = SegformerImageProcessor.from_pretrained("mattmdjaga/segformer_b2_clothes")
model = AutoModelForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")

# Mapeo de etiquetas a nombres
id2label = {
    "0": "Background",
    "1": "Hat",
    "2": "Hair",
    "3": "Sunglasses",
    "4": "Upper-clothes",
    "5": "Skirt",
    "6": "Pants",
    "7": "Dress",
    "8": "Belt",
    "9": "Left-shoe",
    "10": "Right-shoe",
    "11": "Face",
    "12": "Left-leg",
    "13": "Right-leg",
    "14": "Left-arm",
    "15": "Right-arm",
    "16": "Bag",
    "17": "Scarf",
}

# Definir colores específicos para cada etiqueta
label_colors = {
    "0": (0, 0, 0),        # Background - negro
    "1": (255, 0, 0),      # Hat - rojo
    "2": (0, 255, 0),      # Hair - verde
    "3": (0, 0, 255),      # Sunglasses - azul
    "4": (255, 255, 0),    # Upper-clothes - amarillo
    "5": (255, 0, 255),    # Skirt - magenta
    "6": (0, 255, 255),    # Pants - cian
    "7": (192, 192, 192),  # Dress - gris claro
    "8": (128, 0, 0),      # Belt - marrón
    "9": (128, 128, 0),    # Left-shoe - oliva
    "10": (0, 128, 0),     # Right-shoe - verde oscuro
    "11": (128, 0, 128),   # Face - púrpura
    "12": (0, 128, 128),   # Left-leg - turquesa
    "13": (0, 0, 128),     # Right-leg - azul marino
    "14": (255, 165, 0),   # Left-arm - naranja
    "15": (255, 105, 180), # Right-arm - rosa
    "16": (75, 0, 130),    # Bag - índigo
    "17": (0, 255, 127),   # Scarf - verde primavera
}

def segment_image(image):
    # Preprocesar la imagen
    inputs = processor(images=image, return_tensors="pt")

    # Realizar la predicción
    outputs = model(**inputs)
    logits = outputs.logits.cpu()

    # Redimensionar los logits a la tamaño original de la imagen
    upsampled_logits = nn.functional.interpolate(
        logits,
        size=image.size[::-1],
        mode="bilinear",
        align_corners=False,
    )

    # Obtener la segmentación
    pred_seg = upsampled_logits.argmax(dim=1)[0].numpy()

    # Convertir la segmentación en una imagen RGBA
    segmented_image = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.uint8)
    for label_id, color in label_colors.items():
        segmented_image[pred_seg == int(label_id)] = color

    segmented_image = Image.fromarray(segmented_image)

    # Crear una figura para superponer las imágenes
    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.imshow(segmented_image, alpha=0.6)

    # Crear leyenda
    patches = [plt.plot([],[], marker="o", ms=5, ls="", mec=None, color=np.array(color)/255.,
                        label="{:s}".format(id2label[str(i)]))[0]  for i, color in label_colors.items()]
    ax.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize='small')

    # Guardar la imagen con la segmentación superpuesta
    plt.axis('off')
    plt.savefig('segmented_overlay.png', bbox_inches='tight', pad_inches=0)
    segmented_overlay = Image.open('segmented_overlay.png')

    return segmented_overlay

# Crear la interfaz de Gradio
iface = gr.Interface(
    fn=segment_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="pil"),
    title="Segmentación de Prendas de Ropa",
    description="Sube una imagen y el modelo Segformer realizará la segmentación de las prendas de ropa presentes en la imagen.",
)

# Ejecutar la interfaz
iface.launch()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4290b12482ccc514a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
